## Woof vs. Meow - 1

### Building an image classification model using a CNN from scratch

## Data

Data can be downloaded at:
https://www.kaggle.com/c/dogs-vs-cats/data  
All you need is the train set  
The recommended folder structure is:  

### Folder structure

```python
data/
    train/
        dogs/ ### 1024 pictures
            dog001.jpg
            dog002.jpg
            ...
        cats/ ### 1024 pictures
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dogs/ ### 416 pictures
            dog001.jpg
            dog002.jpg
            ...
        cats/ ### 416 pictures
            cat001.jpg
            cat002.jpg
            ...
```
Note : for this example we only consider 2x1000 training images and 2x400 testing images among the 2x12500 available.

The github repo includes about 1500 images for this model. The original Kaggle dataset is much larger. The purpose of this demo is to show how you can build models with smaller size datasets. You should be able to improve this model by using more data.

### Data loading

In [1]:
##This notebook is built around using tensorflow as the backend for keras
!pip install pillow
!KERAS_BACKEND=tensorflow python -c "from keras import backend"

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Using TensorFlow backend.


In [46]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers
import scipy
import pylab as pl
import matplotlib.cm as cm
%matplotlib inline

In [2]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'

### Imports

In [3]:
# used to rescale the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255)

# automagically retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=16,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='binary')

Found 20002 images belonging to 2 classes.
Found 4998 images belonging to 2 classes.


## Small Conv Net

### Model architecture definition

In [4]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [5]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

### Training

In [8]:
nb_epoch = 30
nb_train_samples = 20002
nb_validation_samples = 4998

In [17]:
 model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)

Epoch 1/30
20002/20002 [==============================] - 1656s - loss: 0.5280 - acc: 0.7481 - val_loss: 0.4870 - val_acc: 0.7711
Epoch 2/30
20002/20002 [==============================] - 4211s - loss: 0.4983 - acc: 0.7725 - val_loss: 0.4711 - val_acc: 0.7733
Epoch 3/30
20002/20002 [==============================] - 1741s - loss: 0.4717 - acc: 0.7871 - val_loss: 0.4466 - val_acc: 0.8021
Epoch 4/30
20002/20002 [==============================] - 1479s - loss: 0.4610 - acc: 0.7934 - val_loss: 0.4130 - val_acc: 0.8113
Epoch 5/30
20002/20002 [==============================] - 1676s - loss: 0.4383 - acc: 0.8066 - val_loss: 0.4496 - val_acc: 0.8005
Epoch 6/30
20002/20002 [==============================] - 1613s - loss: 0.4253 - acc: 0.8198 - val_loss: 0.4187 - val_acc: 0.8121
Epoch 7/30
20002/20002 [==============================] - 1518s - loss: 0.4110 - acc: 0.8230 - val_loss: 0.4552 - val_acc: 0.8039
Epoch 8/30
20002/20002 [==============================] - 1537s - loss: 0.4112 - acc: 0.82

In [18]:
model.save_weights('models/basic_cnn_30_epochs.h5')

In [6]:
model.load_weights('models/basic_cnn_30_epochs.h5')

If your model successfully runs at one epoch, go back and it for 30 epochs by changing nb_epoch above.  I was able to get to an val_acc of **0.82** in 30 epochs.

### Evaluating on test set

Computing loss and accuracy :

In [10]:
model.evaluate_generator(validation_generator, nb_validation_samples)

[0.47102052572012998, 0.82032813122864956]

**After 30 epochs the convolutional neural network reaches 82% accuracy. We can witness overfitting, no progress is made over validation set in the next epochs**

This shall be our baseline accuracy, on which we will try and improve in the further models.